In [10]:
import pandas as pd
import numpy as np

from sklearn import svm

from utilidades.validar2 import validar2

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
 svm.SVR

sklearn.svm.classes.SVR

In [3]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

dfinicial = pd.read_csv("data/train_limpio.csv")
dftest = pd.read_csv("data/test_limpio.csv")

dfinicial2 = pd.read_csv("data/esteban/train_ciudades_limpio.csv")
dftest2 = pd.read_csv("data/esteban/test_ciudades_limpio.csv")

xg = pd.read_csv('data/esteban/mejor_corrida.csv')

In [4]:
pd.set_option('display.max_columns', 23) #ver todas las columnas
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

In [5]:
def g_subida2(DFINICIAL, DFTESTEAR, cach, met, d, mi, nomArchivo, pasarAdisco):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
    #ML
    X, y = DFINICIAL.drop( columns = ['precio'] ), dfinicial["precio"]
    #scaling = MinMaxScaler(feature_range=(-1,1)).fit(X)
    #X = scaling.transform(X)
    
    reg = svm.SVR(cache_size = cach, gamma = 'scale', kernel = met, degree = d, max_iter = mi )
    reg.fit(X,y) #entrena como ajustarte a y (precios)
    subida = pd.DataFrame( reg.predict(DFTESTEAR) , columns=["target"]) #crea df con predicciones

    #Formateo
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    #Heurísticas
    # si vale menos de 310 000, ponerle 310 000
    #subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    # si vale más de 12 500 000, ponerle 12 500 000
    #subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    if pasarAdisco:
        subida.to_csv(f"submits/" + nomArchivo, index = False) 
    
    return X, y, subida

In [16]:
%%time

#dfinicial1 = dfinicial
#dftest1 = dftest


dfinicial1 = dfinicial.drop( columns = [ 'anio', 
                            'metrostotales', 
                            'banos',
                            'antiguedad'
                            ]) 


dftest1 = dftest.drop( columns = ['anio', 
                    'metrostotales', 
                    'banos',
                    'antiguedad',
                    ]) 


X, y = dfinicial1.drop( columns = ['precio'] ), dfinicial["precio"] 
scaling = StandardScaler().fit(X)
X2 = scaling.transform(X)
X2 = pd.DataFrame(data = X2, columns = X.columns)
    
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.33, random_state=42)
# y_train son los precios de X_train
# y_test son los precios de X_test
    
reg = svm.SVR( cache_size = 1999, gamma = 'auto', kernel = 'rbf', max_iter = 20000 )
y_train_l = np.log(y_train)
reg.fit(X_train, y_train_l)

pred = reg.predict(X_test)
pred = np.exp(pred)

round(mean_absolute_error(y_test, pred))

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Wall time: 11min 14s


4292705.0

1527121.0